In [ ]:
#!pip install geopandas

  Using cached geopandas-1.1.1-py3-none-any.whl.metadata (2.3 kB)
  Using cached pyogrio-0.11.1-cp312-cp312-macosx_12_0_arm64.whl.metadata (5.3 kB)
  Using cached shapely-2.1.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.8 kB)
Using cached geopandas-1.1.1-py3-none-any.whl (338 kB)
Using cached pyogrio-0.11.1-cp312-cp312-macosx_12_0_arm64.whl (19.5 MB)
Using cached shapely-2.1.2-cp312-cp312-macosx_11_0_arm64.whl (1.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [geopandas]/3 [pyogrio]


In [1]:
import geopandas as gpd
import pandas as pd
import os

In [13]:
# === 1. Ruta base donde están las subcarpetas ===
ruta_base = "/Users/eaha/Documents/TFM/mlops-repo/data/raw/datos_geo_cdmx"

In [14]:
# === 2. Carpeta de salida ===
carpeta_salida = os.path.join("/Users/eaha/Documents/TFM/mlops-repo/data/processed", "csv")
os.makedirs(carpeta_salida, exist_ok=True)

In [15]:
# === 3. Buscar shapefiles en todas las subcarpetas ===
shapefiles = []
for root, dirs, files in os.walk(ruta_base):
    for f in files:
        if f.endswith(".shp"):
            shapefiles.append(os.path.join(root, f))

print(f"🔍 Se encontraron {len(shapefiles)} shapefiles para procesar.")

🔍 Se encontraron 6 shapefiles para procesar.


In [16]:
# === 4. Procesar cada shapefile ===
for shp_path in shapefiles:
    nombre = os.path.splitext(os.path.basename(shp_path))[0]
    print(f"\nProcesando {nombre}.shp ...")

    try:
        # Leer shapefile
        gdf = gpd.read_file(shp_path)

        # Validar geometrías
        gdf = gdf[gdf.is_valid]

        # Calcular centroides (manejo seguro)
        gdf["lon"] = gdf.geometry.centroid.x
        gdf["lat"] = gdf.geometry.centroid.y

        # Convertir a DataFrame sin geometría
        df = gdf.drop(columns="geometry")

        # Nombre de salida (mantiene estructura de subcarpetas)
        subruta_relativa = os.path.relpath(os.path.dirname(shp_path), ruta_base)
        carpeta_destino = os.path.join(carpeta_salida, subruta_relativa)
        os.makedirs(carpeta_destino, exist_ok=True)

        output = os.path.join(carpeta_destino, f"{nombre}_con_coordenadas.csv")
        df.to_csv(output, index=False, encoding="utf-8")

        print(f"✅ Exportado: {output}")

    except Exception as e:
        print(f"⚠️ Error procesando {nombre}: {e}")


Procesando escuelas_privadas.shp ...
✅ Exportado: /Users/eaha/Documents/TFM/mlops-repo/data/processed/csv/escuelas_privadas/escuelas_privadas_con_coordenadas.csv

Procesando hospitales_y_centros_de_salud.shp ...
✅ Exportado: /Users/eaha/Documents/TFM/mlops-repo/data/processed/csv/hospitales_y_centros_de_salud/hospitales_y_centros_de_salud_con_coordenadas.csv

Procesando Metrobus_estaciones.shp ...
✅ Exportado: /Users/eaha/Documents/TFM/mlops-repo/data/processed/csv/mb_shp/Metrobus_estaciones_con_coordenadas.csv

Procesando Metrobus_lineas.shp ...
✅ Exportado: /Users/eaha/Documents/TFM/mlops-repo/data/processed/csv/mb_shp/Metrobus_lineas_con_coordenadas.csv

Procesando STC_Metro_estaciones_utm14n.shp ...
✅ Exportado: /Users/eaha/Documents/TFM/mlops-repo/data/processed/csv/stcmetro_shp/STC_Metro_estaciones_utm14n_con_coordenadas.csv

Procesando STC_Metro_lineas_utm14n.shp ...
✅ Exportado: /Users/eaha/Documents/TFM/mlops-repo/data/processed/csv/stcmetro_shp/STC_Metro_lineas_utm14n_con_co

/var/folders/5x/v8hk2rj9117bkb38sq4yjyfw0000gn/T/ipykernel_9719/563224639.py:14: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["lon"] = gdf.geometry.centroid.x
/var/folders/5x/v8hk2rj9117bkb38sq4yjyfw0000gn/T/ipykernel_9719/563224639.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["lat"] = gdf.geometry.centroid.y
/var/folders/5x/v8hk2rj9117bkb38sq4yjyfw0000gn/T/ipykernel_9719/563224639.py:14: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["lon"] = gdf.geometry.centroid.x
/var/folders/5x/v8hk2rj9117bkb38sq4yjyfw0000gn/T/ipykernel_9719/563224639.py:15: User

Todos los archivos ya tienen las dos columnas que necesitamos: latitud y longitus, lo que ahora tengo que hacer es crear estas dos columnas para el archivo de areas verdes

In [7]:
# === 1. Cargar tu CSV ===
ruta_csv = "/Users/eaha/Documents/TFM/mlops-repo/data/processed/csv/cdmx_areas_verdes_2017.csv"
df = pd.read_csv(ruta_csv)

In [8]:
# === 2. Separar latitud y longitud como strings primero ===
df[["lat", "lon"]] = df["geo_point_2d"].str.split(",", expand=True)

# Convertir a float sin perder precisión (float64)
df["lat"] = df["lat"].astype("float64")
df["lon"] = df["lon"].astype("float64")

In [9]:
# === 3. Mostrar todos los decimales (solo visualización) ===
pd.set_option("display.precision", 12)  # muestra hasta 12 decimales
pd.set_option("display.float_format", lambda x: f"{x:.10f}")

# Verifica que mantiene los valores originales
print(df[["geo_point_2d", "lat", "lon"]].head())

                   geo_point_2d           lat            lon
0  19.4877568178,-99.2060645931 19.4877568178 -99.2060645931
1   19.3887689803,-99.116393168 19.3887689803 -99.1163931680
2  19.3928871121,-99.0869912505 19.3928871121 -99.0869912505
3   19.383106069,-99.1073609486 19.3831060690 -99.1073609486
4  19.3822134465,-99.1028423436 19.3822134465 -99.1028423436


In [10]:
# === 4. Obtener lista de valores únicos en 'subcat_sed' ===
valores_unicos = df["subcat_sed"].dropna().unique()
print("\n🔹 Valores únicos en 'subcat_sed':")
for v in valores_unicos:
    print(f" - {v}")


🔹 Valores únicos en 'subcat_sed':
 - Panteones
 - Deportivos
 - Camellones centrales y laterales
 - Unidades habitacionales
 - Instituciones académicas públicas
 - Plazas
 - Asistencia Social con vegetación CDMX
 - Parques
 - Veg. Arbórea, arbustiva y herbácea de glorietas
 - Vialidades
 - Instituciones académicas privadas
 - Arboledas
 - Instituciones de salud pub/priv
 - Jardines públicos
 - Canales
 - Jardineras públicas y privadas
 - Azoteas verdes
 - Bordos
 - Terrenos baldíos
 - AVA
 - Depresiones orográficas
 - ANP
 - Alamedas
 - Promontorios
 - Colinas
 - Cerros
 - Zona de recarga de manto acuíferos


In [11]:
# === 5. (Opcional) Filtrar por tus etiquetas de interés ===
etiquetas_interes = ["Deportivos", "Unidades habitacionales", "Plazas", "Parques", "Jardines públicos", "Colinas", "Cerros"]
df_filtrado = df[df["subcat_sed"].isin(etiquetas_interes)]

In [12]:
#Guardar
df_filtrado.to_csv("/Users/eaha/Documents/TFM/mlops-repo/data/processed/csv/areas_verdes_filtrado.csv", index=False, encoding="utf-8")

print(f"\n✅ Registros filtrados: {len(df_filtrado)}")


✅ Registros filtrados: 2755


Ahora ya tengo un archivo con las áreas verdes de mi interes.
Exploraré un poco los otros archivos para entenderlos mejor y despues continuaré con el merge por aproximación para obtener la localidad más cercana a cada una de las localizaciones con las cuales contamos

ya estan listos los 6 archivos que necesito para agregar la info a los dos data sets que tenemos.
El de AGEB vs Catastral y el de WebScrapping vs AGEB
Añadiré las locaciones e información de los puntos de servicios e infrestrucutra que son de nuestro interes, haré los merge através de las coordenadas (latittud y longitud) trayendo el punto de mayor cercanía a mis datos

Comenzaré con los archivos donde ya uní el AGEB y el catastral, que está dividido por delegaciones.
Mi_unidad/TFM/join_results_fuzzy (google colab)
entocnes, tendré que hacer un concat de todas ellas y ponerme a realizar los merge
Esto lo ralizaré en el notebook "Deleg_AGEB_Catast.ipynb"

In [ ]:
import duckdb

# Leer todos los archivos parquet en una carpeta
con = duckdb.connect()
df = con.execute("SELECT * FROM 'ruta/a/carpeta/*.parquet'").df()
print(df.head())